 KNN模型的基本原理

KNN是最基本的监督学习算法之一，你只要找出你的“邻居”是什么标签，然后看哪个标签的“邻居”最多，就给这个样本贴上哪个标签。它和“懒惰学习”有点像，因为它在学习的时候并不会做很多工作，只是把数据存起来，等到要分类的时候才来用。它主要是看样本和谁最像，就像人一样，总爱找和自己像的人玩。怎么才算最像呢？就是看各个样本之间的距离，距离越近越像。就像我们平时说的“物以类聚，人以群分”，这个分类方法也是这样，它通过距离来判断样本的相似度。所以，只要找到与测试样本最近的k个训练样本，看这k个样本里哪个类别最多，就认为这个类别是测试样本的类别。

KNN算法的执行流程可以概括为以下步骤：

    计算距离：对于测试样本，计算其与训练集中每个样本的距离，距离的度量方式可以是欧式距离、曼哈顿距离等。
    选择k个最近样本：选择与测试样本距离最近的k个样本。
    投票并返回结果：根据k个最近样本的类别标签进行投票，多数决定原则，即哪个类别标签的多数就选择哪个标签作为测试样本的分类结果。

从上面的过程可以看出，KNN算法的好坏决定性因素有三个：距离的计算方式，K值的选取和数据集。在计算距离时可以采用多种不同的距离衡量方式，例如欧几里得距离、曼哈顿距离、车比雪夫距离等或者其他，用的最多的计算方法是欧几里得距离。既然数据集我们没有办法影响，那k值如何选择好呢？

如果K值太小，那测试样本就只会听“隔壁邻居”的意见，要是这个邻居是个“噪音制造者”，那测试样本的分类就会出错。相反，如果K值太大，那远处的邻居也会插嘴，虽然这样可以让分类更“稳重”，但也可能分得不准确，也就是“没分清楚”。所以，K值得试了才知道，不是我们一开始就能决定的。通常，我们会选一个5-15之间的奇数。但具体选哪个，除了实验和交叉验证，还得看看你的数据量有多大，以及训练集的标签有什么特点。为了加快最近邻居的搜索可以利用KD树进行数据结构的优化。KD树是对数据点在k维空间中划分的一种数据结构。在KD 的构造中，每个节点都是k维数值点的二叉树。既然是二叉树，就可以采用二叉树的增删改查操作，这样就大大提升了搜索效率。

In [17]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
iris = load_iris()
iris_dataframe = pd.concat([pd.DataFrame(iris.data),pd.DataFrame(iris.target)],axis = 1)
x_train ,x_test ,y_train ,y_test  = train_test_split(iris.data,iris.target,test_size=0.3)
def o_distance(a,b):
    return np.sqrt(np.sum(np.square(a-b)**2))
def get_label(x):
    dist = list(map(lambda a:o_distance(a,x),x_train))
    ind = np.argsort(dist)
    '''这是一个排序函数'''
    ind = ind[:k] #选取前 k 个索引，即距离最近的 k 个训练样本。
    labels = np.zeros(3) #初始化一个长度为 3 的数组 labels 用于统计每个类别的出现次数。这里假设我们有三个不同的类别。
    for i in ind:
        label = y_train[i].astype(int)
        labels[label] += 1
    return np.argmax(labels)

In [23]:
def KNN(x_train,y_train,x_test,k):
    def get_label(x):
        dist=list(map(lambda a:o_distance(a,x),x_train))
        
        ind=np.argsort(dist)
        #使用 NumPy 的 argsort 函数来获取按距离从小到大排序后的索引数组。这意味着 ind 数组中存储的不是距离值本身，而是距离值对应的索引即x_train
        ind=ind[:k]
        labels=np.zeros(3)
        for i in ind:
            label=y_train[i].astype(int)
            labels[label]+=1
        return np.argmax(labels) #用于从数组 labels 中找到最大值的索引，这通常用于确定出现次数最多的类别标签
    y_predict=np.zeros(len(x_test))
    for i in range(len(x_test)):
        y_predict[i]=get_label(x_test[i])
    return y_predict
for k in range(1,10):
    y_predict=KNN(x_train,y_train,x_test,k)
    print(accuracy_score(y_test,y_predict))

0.9777777777777777
0.9111111111111111
0.9777777777777777
0.9777777777777777
0.9777777777777777
0.9777777777777777
0.9777777777777777
0.9555555555555556
0.9777777777777777
